# Responsável por tratar os dados pertinente a cliente

## Importando bibliotecas

In [64]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [65]:
import pandas as pd
from src.utils import carregar_tabela_excel
from src.insert_client.insert_client import inserir_cliente

## Importando e tratando os dados

In [66]:
source = "../../data/baseTesteIXC.xlsx"
df = carregar_tabela_excel(source)

In [67]:
# Lista das colunas que fazem sentido enviar para a API
colunas_para_api = [
    'razaoSocial',
    'Email',
    'Celular',
    'cpf_cnpj',
    'rg_ie',
    'idCidade',
    'Endereco',
    'Complemento',
    'Numero',
    'Bairro',
    'Cep',
]

# Filtra o DataFrame para manter apenas essas colunas
df = df[colunas_para_api]


### Tratando CPF_CNPJ e Removendo Duplicados

In [68]:
# Garante que a coluna seja interpretada como string
df["cpf_cnpj"] = df["cpf_cnpj"].astype(str)

# Remove os caracteres '.', '-', '/' usando expressão regular
df["cpf_cnpj"] = df["cpf_cnpj"].str.replace(r"[./-]", "", regex=True)

# Remove duplicados do CPF tratando como um unico cliente
df = df.drop_duplicates(subset=["cpf_cnpj"], keep="first")

### Tratando numero de Celular

In [69]:
# Converte a coluna Celular em string
df["Celular"] = df["Celular"].astype(str)

# Remove os caracteres '(', ')' e '-' usando expressão regular
df["Celular"] = df["Celular"].str.replace(r"[()\-]", "", regex=True)

# Substitui valores 'nan' ou string vazia por '99999999999'
df["Celular"] = df["Celular"].replace(["nan", ""], "99999999999")

### Tratando rg_ie

In [70]:
## Substitui NaN por vazio no campo rg_ie
df["rg_ie"] = df["rg_ie"].fillna("")

### Tratando Complemento, Número e CEP

In [75]:
## Substitui NaN por vazio no campo Complemento
df["Complemento"] = df["Complemento"].fillna("")



## Tratando número da casa
import re

def tratar_numero(valor):
    # Converte para string
    texto = str(valor)

    # Captura todos os dígitos (0-9) presentes no texto
    digits = re.findall(r"\d+", texto)

    if len(digits) > 0:
        # Se encontrar dígitos, junta tudo numa única string
        return "".join(digits)
    else:
        # Se não houver dígitos, retorna "SN"
        return "SN"

# Aplica a função na coluna Numero
df["Numero"] = df["Numero"].apply(tratar_numero)



## Tratando CEP ------------------------
# Converte a coluna Cep em string
df["Cep"] = df["Cep"].astype(str)

# Remove os caracteres '-' usando expressão regular
df["Cep"] = df["Cep"].str.replace(r"[-]", "", regex=True)

In [76]:
inserir_cliente(
    razao="Cliente de Teste",
    cpf_cnpj="12345678900",
    cep="57000000",
    endereco="Rua ABC",
    numero="123",
    bairro="Bairro XPTO",
    cidade="1659",
    uf="AL",
    telefone_celular="82999999999",
    email="teste@example.com",
)

,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,idCidade,Endereco,Complemento,Numero,Bairro,Cep
0,Abelanio Veríssimo Peixoto,radiosnettelecom@gmail.com,28999537513,70380021471,Isento,1659,Vila São Sebastião,Próximo ao cacimbão,0,Centro,57830000
1,Aclecia da Silva,radiosnettelecom@gmail.com,8298155632,12694798497,30234875,1659,Maria de Lurdes,Próximo ao mirante,SN,Centro,57830000
2,Adeilton da Mata,adeiltondamata@gmail.com,8287840401,02301397440,40259595,1659,Ass. Nova Esperança,próximo a casa de farinha,SN,Zona Rural,57830000
3,Adeilton da Silva,radiosnettelecom@gmail.com,82996596910,51629143472,3614522-0,1659,Gravatá,Lote do Sr. Del,SN,Zona Rural,57830000
4,Adeilton Teixeira de Lima,radiosnettelecom@gmail.com,82981674711,00843354402,36145220,1659,João Lyra / Qd: A,,24,Conj. Raimundo Nonato Lopes,57830000
...,...,...,...,...,...,...,...,...,...,...,...
618,Yasmim emanuele correia da Silva,radiosnettelecom@gmail.com,82 9822157025,47574821836,,1659,Rua marechal Floriano Peixoto,próximo ao sindicato,255,Mundo novo,57830000
619,Zaquel da Costa Firmino,betania@gmail.com,82981547921,11741057477,,1659,Ass. Eldorado dos Carajás,mãe da Betânia,SN,Zona Rural,57830000
620,Zélia Maria da Silva Oliveira,radiosnettelecom@gmail.com,82981478041,10048396427,,1659,platô 3/ Q: 10,,37,Conj. Raimundo Nonato Lopes,57830000
621,ZENILDA DE LIMA FREIRE,radiosnettelecom@gmail.com,82982297108,04430401411,,1659,Assentamento Flor do Mundau,irmã do dimas,SN,Zona Rural,57830000
